In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
rules_df = pd.read_csv(r"./rules.csv")
display(rules_df.head())
rules_df.info()

,rule
0,100242812>80361758
1,100242812>89031406
2,1003153762>1016449477
3,1006024995>2727415265
4,1006024995>866012366


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14238 entries, 0 to 14237
Data columns (total 1 columns):
rule    14238 non-null object
dtypes: object(1)
memory usage: 111.4+ KB


In [3]:
association_df = pd.read_csv(r"./association_order.csv")
display(association_df.head())
association_df.info()

,orderid,itemid
0,31379820545759,719740607
1,31378575577269,1825360194
2,31369591568249,1108903291
3,31369836201769,4507360843
4,31372360246729,1821888475


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423861 entries, 0 to 423860
Data columns (total 2 columns):
orderid    423861 non-null int64
itemid     423861 non-null int64
dtypes: int64(2)
memory usage: 6.5 MB


In [4]:
df_byorder = association_df.groupby('orderid')['itemid'].apply(set).reset_index(name = 'item_set').set_index('orderid')
df_byorder['count_item'] = df_byorder['item_set'].apply(len)
df_byorder.head()

,item_set,count_item
orderid,,
31338000509845,"{2255381856, 2662778119, 1825056968, 680760215...",9
31338001158725,"{5013004529, 6112866826, 5712867361}",3
31338002167561,"{5712106216, 3310344299, 3310738830, 380238459...",6
31338004895579,"{911192064, 760247001, 1142881028, 5410312558}",4
31338005995334,"{5305001639, 3700032699, 6900080063}",3


In [5]:
df_byitem = association_df.groupby('itemid')['orderid'].apply(set).reset_index(name = 'order_set').set_index('itemid')
df_byitem['count_order'] = df_byitem['order_set'].apply(len)
df_byitem.head()

,order_set,count_order
itemid,,
103572,{31347710941008},1
103580,{31338334007720},1
108696,{31372952019978},1
240094,{31380092284102},1
262532,"{31362741095032, 31379924542443}",2


In [6]:
rules_df['hyp'] = rules_df.rule.apply(lambda x: x.split('>')[0].split('&')) # Hyphotesis
rules_df['con'] = rules_df.rule.apply(lambda x: x.split('>')[-1].split('&')) # Conclusion
rules_df.head(10)

,rule,hyp,con
0,100242812>80361758,[100242812],[80361758]
1,100242812>89031406,[100242812],[89031406]
2,1003153762>1016449477,[1003153762],[1016449477]
3,1006024995>2727415265,[1006024995],[2727415265]
4,1006024995>866012366,[1006024995],[866012366]
5,1006149508>2867088619,[1006149508],[2867088619]
6,1006149508>599643179,[1006149508],[599643179]
7,1006149508>943964984,[1006149508],[943964984]
8,1008435393&1008455319>454021998,"[1008435393, 1008455319]",[454021998]
9,1008435393&1336146681>454021998,"[1008435393, 1336146681]",[454021998]


In [7]:
N_data = association_df.shape[0]
def calculate_confidence(x, df_byitem, df_byorder):
    for i, xh in enumerate(x['hyp']):
        if i == 0:
            set_h = df_byitem['order_set'][int(xh)]
        else:
            set_h = set_h.intersection(df_byitem['order_set'][int(xh)])
    prob_h = len(set_h)
    count_hc = 0
    for order in set_h:
        cond = 0
        for xc in x['con']:
            if int(xc) in df_byorder['item_set'][order]:
                cond += 1
        if cond == len(x['con']):
            count_hc += 1
    prob_hc = count_hc
    
    prob_cgivenh = prob_hc/prob_h
    return prob_cgivenh*1000
    
rules_df['confidence'] = rules_df.apply(lambda x: calculate_confidence(x, df_byitem, df_byorder), axis=1)
rules_df.head(10)

,rule,hyp,con,confidence
0,100242812>80361758,[100242812],[80361758],470.588235
1,100242812>89031406,[100242812],[89031406],352.941176
2,1003153762>1016449477,[1003153762],[1016449477],388.888889
3,1006024995>2727415265,[1006024995],[2727415265],171.428571
4,1006024995>866012366,[1006024995],[866012366],171.428571
5,1006149508>2867088619,[1006149508],[2867088619],333.333333
6,1006149508>599643179,[1006149508],[599643179],333.333333
7,1006149508>943964984,[1006149508],[943964984],388.888889
8,1008435393&1008455319>454021998,"[1008435393, 1008455319]",[454021998],352.941176
9,1008435393&1336146681>454021998,"[1008435393, 1336146681]",[454021998],818.181818


In [8]:
df_byitem.count_order.sum()+df_byorder.count_item.sum(), N_data*2

(847722, 847722)

In [9]:
submission=rules_df[['rule','confidence']]
submission.head()

,rule,confidence
0,100242812>80361758,470.588235
1,100242812>89031406,352.941176
2,1003153762>1016449477,388.888889
3,1006024995>2727415265,171.428571
4,1006024995>866012366,171.428571


In [10]:
# submission['confidence'] = submission['confidence'].round()

In [15]:
submission['confidence'] = submission['confidence'].astype('int')

C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14238 entries, 0 to 14237
Data columns (total 2 columns):
rule          14238 non-null object
confidence    14238 non-null int32
dtypes: int32(1), object(1)
memory usage: 167.0+ KB


In [17]:
submission.to_csv('market(1).csv', index=False)

In [18]:
submission.head()

,rule,confidence
0,100242812>80361758,470
1,100242812>89031406,352
2,1003153762>1016449477,388
3,1006024995>2727415265,171
4,1006024995>866012366,171
